# Genomics Algorithmics project

## Development of a mapping solution over a reference genome for sequencing datas  

### Nicolas Parisot & Sergio Peignier

## Reading sequencing datas with *Biopython* package

### Genome and inversed genome of *Plasmodium falciparum*

With *Biopython*, each of the 15 *Plasmodium falciparum* chromosomes is stored in a python list under the form of a string. The python list is named **chromosomes**. Mapping operations will be realised on these chromosomes.

In [120]:
from Bio import SeqIO
from tqdm import tqdm
from Chromosome import Chromosome
from dc3 import dc3
import bwt
import tools
import mapping
import importlib
import numpy as np
import cProfile

To store all the result, a Chromosome class was created (check Chromosome.py file) to manage easily all the data.
So for each kind of data, a file has been saved to the corresponding chromosome (when a "-" is in front of the number, it mean it's the inversed complementary of the chromosome).
In the following chuck, all needed data has instanced.

In [3]:
chromosomes:list[Chromosome] = []
chromosomes_inv:list[Chromosome] = []

with tqdm(total=30, desc="Chromosomes importation") as pbar:
    for i,record in enumerate(SeqIO.parse("SEQUENCES/P_fal_genome.fna", format="fasta")):
        # Normal chromosomes
        name = f"P_fal_chromosome_{i+1}"
        chromo = Chromosome(name,record.seq,record.id)
        chromosomes.append(chromo)
        pbar.update(1)
        
        # Inversed chromosomes
        name_inversed_comp = f"P_fal_chromosome_-{i+1}"
        inversed_comp_seq = tools.inverse_sequence(record.seq)
        chromo_inv = Chromosome(name_inversed_comp,inversed_comp_seq,record.id)
        chromosomes_inv.append(chromo_inv)
        pbar.update(1)

Chromosomes importation: 100%|██████████| 30/30 [00:25<00:00,  1.20it/s]


Number and lengths of the chromosomes are given here:

In [4]:
print(f"Chromosomes number: {len(chromosomes)}")
print("Chromosomes length, from 1 to 15:")
for i, chromo in enumerate(chromosomes):
    i += 1
    print(f"Length of the chromosome {i:>2} : {len(chromo.DNA_dol):<7} pairs of bases")
for i, chromo in enumerate(chromosomes_inv):
    i += 1
    print(f"Length of the chromosome -{i:>2} : {len(chromo.DNA_dol):<7} pairs of bases")

Chromosomes number: 15
Chromosomes length, from 1 to 15:
Length of the chromosome  1 : 640852  pairs of bases
Length of the chromosome  2 : 947103  pairs of bases
Length of the chromosome  3 : 1067972 pairs of bases
Length of the chromosome  4 : 1200491 pairs of bases
Length of the chromosome  5 : 1343558 pairs of bases
Length of the chromosome  6 : 1418243 pairs of bases
Length of the chromosome  7 : 1445208 pairs of bases
Length of the chromosome  8 : 1472806 pairs of bases
Length of the chromosome  9 : 1541736 pairs of bases
Length of the chromosome 10 : 1687657 pairs of bases
Length of the chromosome 11 : 2038341 pairs of bases
Length of the chromosome 12 : 2271495 pairs of bases
Length of the chromosome 13 : 2925237 pairs of bases
Length of the chromosome 14 : 3291937 pairs of bases
Length of the chromosome 15 : 34251   pairs of bases
Length of the chromosome - 1 : 640852  pairs of bases
Length of the chromosome - 2 : 947103  pairs of bases
Length of the chromosome - 3 : 1067972 p

The objectives of this project is to perform a **mapping** of short DNA sequences (1500000 of 100 nucleotides long) over all the chromosomes we just imported. In other words, we need to find their localisation.

Due to the length of the genome and the number of read to map, this problem can't be done with "naive" methods. Instead, we use a **string-search** algorithm, based on the Burrows Wheeler Transform (BWT) (Sources: https://www.molgen.mpg.de/3708260/bwt_fm.pdf). The read are mapped over the BWT of all the chromosomes. To that extend, we need to compute the BWT of very large strings which can be very time consuming. The **DC3 algorithm** allows us to compute very efficiently suffix array of strings and the we use it to compute the BWT.

_____________________

## Suffix array of the genome with DC3 algorithm
The following lines of code are used to import the suffix table computed from the DC3 algorithm. Suffix tables are stored in npy files, one of the best way of storage for list <p>
Source: https://stackoverflow.com/questions/9619199/-way-to-preserve-numpy-arrays-on-disk
<p>
To compute the suffix array efficiently, a DC3 algorithm has done, and then reducing the time complexity to O(n). To see a explanation, you can check the dc3.py.
All result are saved in the RESULTS directory. Can be read by using the **Chromosome.import_dc3_result** method

In [5]:
for chromo in chromosomes+chromosomes_inv:
    print(chromo.file_name)
    if chromo.suffix_table is None:
        dc3result = dc3(chromo.DNA_dol)
        chromo.export_dc3_result(dc3result)

P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


## Burrows Wheeler Transform (BWT) of the genome
This chunk creates (or imports) the BWT transform of every chromosomes. The algorithm and its explaination can be find on the bwt.py file, using also a O(n) time complexity. 
All result are saved in the RESULTS directory. Can be read by using the **Chromosome.import_bwt_result** method

In [6]:
for chromo in chromosomes+chromosomes_inv:
    if chromo.bwt is None:
        bwt_result = bwt.bwt(str(chromo.DNA_dol),chromo.suffix_table)
        chromo.export_bwt_result(bwt_result)


P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


Next we compute or import the rank matrix of each chromosome. Rank matrices are large python dictionnaries storing for a chromosome, the rank array of each of its four nucleotides "A", "C", "G", "T" and for the end of string character "$".<p>
Rank matrices are very useful for our *string_search* function in the mapping module. They allow us to localise substrings with $O(1)$ complexity (again the source: https://www.molgen.mpg.de/3708260/bwt_fm.pdf).

In [7]:
for chromo in chromosomes+chromosomes_inv:
    if chromo.rank_mat is None:
        print(len(chromo.bwt))
        rank_mat = bwt.create_rank_mat(chromo.bwt)
        chromo.export_rank_matrix_result(rank_mat) 

P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


## Reads importation
*Reads* sequence and identifier are extracted with *Biopython* and stored in a python list **reads**. The sequencing produced 100000 *reads* for each of the 15 chromomoses for a total of 1500000 *reads*. All the *reads* have the same length: 100 pairs of bases. 

In [4]:
reads = []
with tqdm(total=1500000, desc="Reads importation") as pbar:
    for record in SeqIO.parse("SEQUENCES/P_fal_reads.fq", format="fastq"):
        reads.append((str(record.seq), record.id))
        pbar.update(1)

Reads importation: 100%|██████████| 1500000/1500000 [00:25<00:00, 58423.70it/s]


In [21]:
print(f"Number of reads: {len(reads)}")
print(f"Reads length: {len(reads[0][0])}")

Number of reads: 1500000
Reads length: 100
<class 'str'>


## Mapping algorithm

The mapping algorithm is working thanks to a string search algorithm using the bwt. It work with a rank matrice, wich calculate the sum of a char for a position given is the sequence. With this method, you can simulate the hand made algorithm with bwt, using the **first colomn of bwt list** and the **last one**. Then checking for the next char, if it's still following thanks to the rank matrix. If you wan more precision about the algorithm, you can check it in the mapping.py file.

To find the location(s) of a given read on a genome, we start with the cutting of a read into smaller pieces, called **kmers**.
Then we find all the locations of these kmers and we "reconstruct" the read to find it's most precise localisation. <p>
The length of the kmers is set to 20 which. It seems to be a good trade-off between precision and mapping speed (the smaller a kmer is, the better the mapping can be because it is less sensitive to mutations).<p>
To start, no error is allowed, that mean that we search the perfect copy of the read in the chromosome sequence. <p>
To gain time, as it's precise in the reads file, we map the read only for the corresponding chromosome (so, 100.000 per chromosomes). So we apply our algorithm for the 15 chromosomes and the same thing for all the inversed complementary chromosomes

In [118]:
importlib.reload(mapping)

with tqdm(total=1500000, desc="Reads importation") as pbar:
    for chr_n in range(len(chromosomes)):
        iddsq = 0
        starts_reads = chr_n*100000
        end_reads = ((chr_n+1)*100000)
        pos_dict = {}
        inv_pos_dict = {}
        for i in range(starts_reads,end_reads):
            read = str(reads[i][0])
            k_reads_locs = mapping.string_search(read,chromosomes[chr_n])
            if k_reads_locs.size > 0:
                if k_reads_locs[0] >= 0:
                    pos_dict[read] = k_reads_locs
            k_reads_locs = mapping.string_search(read,chromosomes_inv[chr_n])
            if k_reads_locs.size > 0:
                if k_reads_locs[0] >= 0:
                    inv_pos_dict[read] = k_reads_locs
            pbar.update(1)
        np.save(f"RESULTS/MAPPING_RESULT_d={0}_{chr_n}.npy",pos_dict)
        np.save(f"RESULTS/MAPPING_RESULT_d={0}_-{chr_n}.npy",inv_pos_dict)


Reads importation: 100%|██████████| 1500000/1500000 [03:01<00:00, 8244.34it/s] 


For each chromosomes, the result of mapping cotainnig all sequence for each read is saved in a .npy files. The next chuck show a example for the first chromosome.

In [124]:
print(np.load(f"RESULTS/MAPPING_RESULT_d=0_0.npy",allow_pickle=True))

{'TCATGATTTACATATATTTGTAAAACATATATAATCTGTCCAGACATATTATATAATTGATAATATAATATATATATATATATATAAATTATTACTTCTC': array([471737], dtype=int64), 'ATAAAATAATAAACTACAACTAAACATTACATATTTATATATGTCACCATTTCTTTATAGGTATATAATCAAAGGAACTATTACATTACACCACGTCATA': array([592074], dtype=int64), 'TTACGTATATATCATTTGTCATATATATAATATAAATTTATATTATTATATAAAAAATTTTTATCATGATATATAATATAATATAATATAATAATACAAT': array([270169], dtype=int64), 'ATGTATCATAATACTTTATAAAAGCATAAATACAATTTTTCATTTCATCAGTTGATTTTCCATCTTTAACTAATTTATAAAAATTAAGAGTATGTTCAAT': array([56757], dtype=int64), 'TTACGATATAGTGGAATATACCAAATCATTTTGTAATACGTGGACAGATAGACACAATTATATGTTAGAAAAATGGAATAAAGAAAATTGGTTTGTTCAA': array([580626], dtype=int64), 'ATTTTTACGTAAAAAAACAGATATAATATTCCCAAAAAAGATATAATAATAAATTTAATTAATATAGATCTTAAAAATAGGTTTTTTTTTTTTTGTATCA': array([121139], dtype=int64), 'TTTTATTTTTATTTTTTGTTCTGCTTACCGTTGTAGGATATGGAACAACTTAAGAACCGCATAAACAACCTGAACATGCTAGCCGTCGTATATTTAGCAT': array([418998], dtype=int64), 'ATCATAAGTTTTATTGTTTTTTTTGATATATATTCCTTTGAAAAACATATGAGGTATTTAA

### Complexity study

A way to try the complexity of this algorithm is to use Profiling program (cProfile). But we can study directly the algorithm complexity: it only parse a len of the kmer, thanks to the use of rank matrix. So a time complexity of O(n) but with n really small in comparaison of the length of the DNA. So the time complexity mostly depend to the lenght of the DNA. But also to the number of read, causing a big number of function used, and then costing a big amount of time. But also because we have generated a bwt and a suffix array with a O(n) complexity, with n the DNA sequence size and the search sequence complexity is only O(1).
<p>
To reduce the quantity of data, we can also clean cleaning method as Trommomatic. Indeed we know that extremity of reads contains bad quality bases. Thats why using some methods of cleaning can help us to gain in performence. 

In [125]:
def complexity_test(): #In a function to use cProfile
    with tqdm(total=1500000, desc="Reads importation") as pbar:
        for chr_n in range(len(chromosomes)):
            iddsq = 0
            starts_reads = chr_n*100000
            end_reads = ((chr_n+1)*100000)
            pos_dict = {}
            inv_pos_dict = {}
            for i in range(starts_reads,end_reads):
                read = str(reads[i][0])
                k_reads_locs = mapping.string_search(read,chromosomes[chr_n])
                if k_reads_locs.size > 0:
                    if k_reads_locs[0] >= 0:
                        pos_dict[read] = k_reads_locs
                k_reads_locs = mapping.string_search(read,chromosomes_inv[chr_n])
                if k_reads_locs.size > 0:
                    if k_reads_locs[0] >= 0:
                        inv_pos_dict[read] = k_reads_locs
                pbar.update(1)

cProfile.run("complexity_test()")

Reads importation: 100%|██████████| 1500000/1500000 [02:55<00:00, 8549.44it/s]

         12482234 function calls in 175.462 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    6.466    6.466  175.452  175.452 4092734919.py:1(complexity_test)
   738916    1.172    0.000    6.675    0.000 <__array_function__ internals>:177(sort)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.010    0.010  175.462  175.462 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:48(create_string_buffer)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:111(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:17(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:21(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:27(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:53(_

### Mapping but allowing error on middle kmer

Now, we map only the first and last kmer, and check if the distance between the 2 kmers is equal to the theoretical value. And then we test again for each kmer until we reach our max number of error allowed. If it is not reaching, it mean that we can accept the position of the initial kmer as the position of the read. As a exemple, here a number of error allowed is 1, meaning that one kmer of five, (but really 3, because the first and the last one have to be 100% mapped) can contain a mutation or a sequencing error. It's these value because we chose 20 as k to make the kmer.

In [117]:
importlib.reload(mapping)

nb_of_error_allowed = 1
def cProfile_test():
    with tqdm(total=3000000, desc="Reads importation") as pbar:
        for chr_n in range(len(chromosomes)):
            iddsq = 0
            starts_reads = chr_n*100000
            end_reads = ((chr_n+1)*100000)
            pos_dict = {}
            inv_pos_dict = {}
            for i in range(starts_reads,end_reads):
                read = str(reads[i][0])
                k_reads = mapping.cut_read_to_kmer(read,20)
                k_reads_locs = [mapping.string_search(k_read,chromosomes[chr_n]) for k_read in k_reads]
                k_reads_locs_inv = [mapping.string_search(k_read,chromosomes_inv[chr_n]) for k_read in k_reads]
                res = mapping.link_kread(k_reads_locs, 20, nb_of_error_allowed)
                if len(res_inv) > 0:
                        pos_dict[read] = res
                res_inv = mapping.link_kread(k_reads_locs_inv, 20, nb_of_error_allowed)
                if len(res_inv) > 0:
                    inv_pos_dict[read] = res_inv
                pbar.update(1)
            np.save(f"RESULTS/MAPPING_RESULT_d={nb_of_error_allowed}_{chr_n}.npy",pos_dict)
            np.save(f"RESULTS/MAPPING_RESULT_d={nb_of_error_allowed}_-{chr_n}.npy",inv_pos_dict)
        
cProfile.run("cProfile_test")



Reads importation:  50%|████▉     | 1499985/3000000 [59:31<59:31, 420.01it/s]   


## Comparison and verification

Thanks to this algorithm we obtained a result file giving us a position for each reads (when there is atleast a correspondence with **d** error allowed). We can then compare to a SAM file of all read mapped (first BAM but the converted to SAM to be readable for human). We can then compare our result to the theorical result to test our algorithm perfomence. But because of lack of time, this couldn't be done. But we could do it to only comparing the localization that the sam file give, and compare it to our localization. And even to be sure, to verify directelty with the localization on the DNA sequence.

## To conclude

We have not yet a optimized algorithm (need atleast 1 hour to make everything from scratch on a not really computer). But some strategy could be done to gain time, like using subprocessing allowing to use multiple core of the precessor. Or just work on the algorithm by checking what takes the more time thanks to cProfile. This long duration of computing is maybe the main difficulty, because it become hard to verify our algorithm on reel data.